In [1]:
from langchain_ollama import OllamaLLM



MODEL = "llama3.1"
model = OllamaLLM(model=MODEL,
                  temperature=0)



In [3]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser
chain.invoke("tell me a joke")

ResponseError: 

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("HandsOnMl.pdf")
pages = loader.load_and_split()

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question below using the context. if you don't know the answer reply "I Don't Know"

question = {question}
context = {context} 
"""
prompt = PromptTemplate.from_template(template)
print(prompt.format(context = "You are a wizard", question = "what are you?"))


Answer the question below using the context. if you don't know the answer reply "I Don't Know"

question = what are you?
context = You are a wizard 



In [8]:
chain = prompt | model | parser

chain.invoke({
    "context" : "you are a dog",
    "question" : "what are you?"
})

ResponseError: 

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(pages, embeddings)

c:\Projects\Rag_Pdf\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


ValueError: Error raised by inference API HTTP code: 404, {"error":"model \"llama2\" not found, try pulling it first"}